## <div style="background-color: #BAE3BC; padding: 10px;"><b>Exercicis de Web Scraping.</b>

</div>

- ### Exercici 1

Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

Triarem fer l'scraping de "Quotes to Scraping" i de "Wikipedia". Començarem per la pàgina web "http://quotes.toscrape.com"

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 1. Scraping amb Beautiful Soup - http://quotes.toscrape.com

Primer de tot, explorarem la web.

In [2]:
URL = "http://quotes.toscrape.com"
page = requests.get(URL)

#print(page.text)


Una altre manera de comprovar que la nostra sol·licitud per accedir a les dades de la pàgina web hagi estat satisfactòria, és a través del codi d'estat.

- Si ens dona "200" esta tot "*ok*".
- Si ens dona "404" és que hi ha "*Not found*".
- Si ens dona "500" és que hi ha "*several internal error*".

In [3]:
page.status_code

200

Ara crearem un objecte Beautiful Soup i li pasarem dos arguments (content i html.parser). Passarem de text a content per evitar problemes amb la codificació de caràcters. D'una altre banda posarem html.parser per assegurar-nos de fer servir l'analitzador adequat per al contingut HTML.

In [4]:
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(class_='container')

Una manera de mostrar el contingut de la web de manera més clara posant tota la info dins del <div> és fer-ho a través d'un *prettify*.

In [5]:
#print(results.prettify())

Farem servir novament el *find*, però ara amb una cerca específica del títol de tot el contingut de la web.

In [6]:
print(soup.find('title').string)

Quotes to Scrape


Continuarem buscant les class quote a través de totes les div on es trobi continguda. Així podrem saber quantes cites hi han a la primera pàgina de la web.

In [7]:
quotes_n = results.find_all("div", class_="quote", itemscope="")
len(quotes_n)

10

A continuació farem un bucle sobre l'informació d'interés dins dels elements de les *quotes*, és a dir, extraurem el "Text", "l'Autor" i les seves paraules clau o "Tags".

In [8]:
for quote_n in quotes_n:
    cita = quote_n.find("span", class_="text")
    autor = quote_n.find("small", class_="author")
    p_clau = quote_n.find("div", class_="tags")
    print(cita.text.strip())
    print(autor.text.strip())
    print(p_clau.text.strip())
    print()

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
Tags:
            
change
deep-thoughts
thinking
world

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
Tags:
            
abilities
choices

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
Tags:
            
inspirational
life
live
miracle
miracles

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
Tags:
            
aliteracy
books
classic
humor

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
Tags:
            
be-yourself
inspirational

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
Tags:
            
adulthood
success
value

### 1.2 Scraping amb Selenium - http://quotes.toscrape.com

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

- 1 Amb *Selenium* primer de tot hem de posar el navegador que farem servir.
- 2 Després, obtindrem la direcció URL. 
- 3 I finalment crearem objectes on posarem els elements volem obtenir.

In [10]:
#1
driver = webdriver.Chrome()

#2
driver.get("http://quotes.toscrape.com/")

#3
cites = driver.find_elements(By.CLASS_NAME, "text")
autors = driver.find_elements(By.CLASS_NAME, "author")
p_claus = driver.find_elements(By.CLASS_NAME, "tags")


A continuació farem un bucle on recorrerem cada *quote* de la web i així obtindrem totes les cites.

In [11]:
for cita_n in cites:
    cita_n = cita_n.text.strip()
    print(cita_n)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”


Ara farem el mateix però amb els autors. 

In [12]:
for autor_n in autors:
    autor_n = autor_n.text.strip()
    print(autor_n)

Albert Einstein
J.K. Rowling
Albert Einstein
Jane Austen
Marilyn Monroe
Albert Einstein
André Gide
Thomas A. Edison
Eleanor Roosevelt
Steve Martin


I per acabar extraurem les paraules clau:

In [13]:
for p_claus_n in p_claus:
    p_claus_n = p_claus_n.text.strip()
    print(p_claus_n)

Tags: change deep-thoughts thinking world
Tags: abilities choices
Tags: inspirational life live miracle miracles
Tags: aliteracy books classic humor
Tags: be-yourself inspirational
Tags: adulthood success value
Tags: life love
Tags: edison failure inspirational paraphrased
Tags: misattributed-eleanor-roosevelt
Tags: humor obvious simile


Tanquem el navegador

In [14]:
driver.close()

### 1.3 Wikipedia amb Beautiful Soup - [Wikipedia Valle de la luna](https://es.wikipedia.org/wiki/Valle_de_la_Luna_(Chile))

En aquesta oportunitat farem un codi resumit on extraurem el contingut principal dels paràgrafs de la descripció del "valle de la luna" segons la Wikipedia. Això ho farem fent servir un condicional que recorrerà tot el procés, des de la crida de la URL fins l'obtenció de les parts del text a través del tag "p". A més a més, hem de dir que començarem el codi revisant que la sol·licitud ha estat reeixida.

In [15]:
# sol·licitud http a la web desitjada"
url = "https://es.wikipedia.org/wiki/Valle_de_la_Luna_(Chile)"
page_wiki = requests.get(url)

# Verificació sol·licitud 
if page_wiki.status_code == 200:
    # obtenció contingut HTML
    html = page_wiki.content

    # Objecte Beautiful Soup per analitzar l'HTML
    soup_wiki = BeautifulSoup(html, 'html.parser')

    # Cerca contenidor principal
    text_principal = soup_wiki.find('div', {'id': 'mw-content-text'})

    # Obtenció paràgrafs contingut principal
    parts_text = text_principal.find_all('p')

    # Imprimir contingut
    for part in parts_text:
        print(part.get_text())
else:
    print("La sol·licitud no ha estat reeixida.")

El valle de la Luna es un paraje desértico y punto de atracción turística, ubicado en el desierto de Atacama, a 13 km al oeste de San Pedro de Atacama y a 110 km al SE de Calama, perteneciente a la región de Antofagasta. En 1982 fue declarado santuario de la naturaleza y es parte de la reserva nacional Los Flamencos, su ingreso está administrado por la Conaf y es accesible desde San Pedro de Atacama.

Junto con el llamado valle de la Muerte, este lugar es único dentro del entorno que rodea la sierra Orbate a 2500 m s. n. m., se trata de una depresión de tamaño relativo de 440 km² de la llamada cordillera de la Sal en la cuenca del salar de Atacama circundado por crestas de 500 m de elevación. Su extensión abarca desde el oeste por la cordillera de Domeyko, por el este por Toconao y Peine por el sur.[1]​

Estudios geológicos recientes indican que existió un gran lago o mar interior perimetral en el sector El Tambo entre Toconao y Peine a finales de la Era Terciaria formada por la cuenca

Ara anem a buscar els títols dels paràgrafs anteriors

In [16]:
titols = [title.get_text().replace("[editar]", "") for title in text_principal.find_all('h2')]

for titol in titols:
    print(titol)

Génesis geológica
Características
Clima
Cultura popular
Enlaces externos
Véase también
Referencias


### 1.4 Wikipedia amb Selenium - [Wikipedia Valle de la luna](https://es.wikipedia.org/wiki/Valle_de_la_Luna_(Chile))

Per trobar l'informació que necessitem però amb la llibreria Selenium, farem servir una estructura similar a la feta amb Beautiful Soup, però per descomptat, utilitzant els mètodes corresponents a aquesta llibreria.

In [17]:
# Inicilitzant el controlador
driver_wiki = webdriver.Chrome()

# navegant a la pàgina web
driver_wiki.get("https://es.wikipedia.org/wiki/Valle_de_la_Luna_(Chile)")

# Crear objectes per cercar paràgrafs i títols
parts_text_wiki = driver_wiki.find_elements(By.TAG_NAME, "p")
titols_wiki = driver_wiki.find_elements(By.TAG_NAME, "h2")

# Imprimir el contingut del paràgrafs
for part in parts_text_wiki:
    print(part.text)
    print()

El valle de la Luna es un paraje desértico y punto de atracción turística, ubicado en el desierto de Atacama, a 13 km al oeste de San Pedro de Atacama y a 110 km al SE de Calama, perteneciente a la región de Antofagasta. En 1982 fue declarado santuario de la naturaleza y es parte de la reserva nacional Los Flamencos, su ingreso está administrado por la Conaf y es accesible desde San Pedro de Atacama.

Junto con el llamado valle de la Muerte, este lugar es único dentro del entorno que rodea la sierra Orbate a 2500 m s. n. m., se trata de una depresión de tamaño relativo de 440 km² de la llamada cordillera de la Sal en la cuenca del salar de Atacama circundado por crestas de 500 m de elevación. Su extensión abarca desde el oeste por la cordillera de Domeyko, por el este por Toconao y Peine por el sur.1

Estudios geológicos recientes indican que existió un gran lago o mar interior perimetral en el sector El Tambo entre Toconao y Peine a finales de la Era Terciaria formada por la cuenca hi

In [18]:
for titol in titols_wiki:
    print(titol.text.replace("[editar]", ""))
    print()

Contenidos

Génesis geológica

Características

Clima

Cultura popular

Enlaces externos

Véase también

Referencias



In [19]:
# Tancant el controlador de selenium
driver.quit()

- ### Exercici 2

Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

[Kaggel datasets Football](https://www.kaggle.com/datasets/vivovinco/20212022-football-team-stats)

*Títol*: Quotes to Scrape

*Context*:

“Quotes to Scrape” és un lloc de la web que serveix per practicar web scraping. La web conté cites de gent molt reconeguda. Aquestes cites porten el nom de qui les ha dit i també paraules clau que se extreuen directament de la mateixa cita.

*Contingut del dataset*:

1.	10 cites de persones molt reconegudes
2.	Nom de l’autor
3.	Paraules clau
---------------------------------------------------------------------------------------------------------------------------
*Títol*: Wikipedia – Valle de la Luna (Chile)

*Context*:

“Wikipedia” és una enciclopèdia lliure, políglota i editada de manera col·laborativa. És adminstrada per la fundació Wikimedia, una organització sense ànim de lucre el finançament del qual està basada en donacions.

Dins d’aquest lloc web s’ha fet una cerca d’un dels llocs més turístics de Xile, “El Valle de la Luna”. A conitnuació es descriuré l’scraping fet amb els resultats obtinguts.

*Contingut del dataset*:

1.	Text introductori + text de les diferents característiques del Valle de la Luna. Contingut en general.

2.	Els títols del contingut:

- Gènesi Geològica
- Característiques
- Clima
- Cultura popular
- S’hi afegeixen els títols (sense contingut) de: Enllaços externs -  Vegeu També - Referències
---------------------------------------------------------------------------------------------------------------------------
*Data de creació dels datasets*:
08/06/2023

*Hora de l’extracció*:
19:45 pm
